# Homework 1 - Answer Sheet

__Name__: Dong Wang

__UID__: 804363328

## Problem 1 Multinomial Naive Bayes

Please write down your answer in the cell below.

The log likelihood function:
$logL(\Theta)=log \prod\limits_d p(x_d,y_d|\Theta)=\sum\limits_d log[p(x_d|y_d)p(y_d)]=\sum\limits_d (x_{dn}log \beta_{y_d n}+log \pi_{y_d})$   
The optimization problem: 
$$\max\limits_\Theta log L(\Theta) \  s.t. \sum\limits_j \pi_j=1 \  and \  \sum\limits_n \beta_{jn}=1$$  
1.Solve $\hat{\pi_j}$:  
$$J_1=\sum\limits_d log\pi_{y_d}+\lambda_1(\sum\limits_j \pi_j-1)=\sum\limits_d \sum\limits_j \mathbb{1}(y_d==j) log\pi_j+\lambda_1(\sum\limits_j \pi_j-1)$$  
$$\frac{\partial J_1}{\partial \pi_j}=\sum\limits_d \mathbb{1}(y_d==j) \frac{1}{\pi_j}+\lambda_1 =0\\  
\Rightarrow \sum\limits_d\sum\limits_j \mathbb{1}(y_d==j)+\lambda_1 \sum\limits_j\pi_j)= \sum\limits_d \sum\limits_j \mathbb{1}(y_d==j)+\lambda_1=0 \\ \Rightarrow \lambda_1=-\sum\limits_j\sum\limits_d \mathbb{1}(y_d==j)$$ 
So we will have: $\hat{\pi_j}=-\frac{\sum\limits_d \mathbb{1}(y_d==j)}{\lambda_1}= \frac{\sum\limits_d \mathbb{1}(y_d==j)}{\sum\limits_d \sum\limits_j \mathbb{1}(y_d==j)}=\frac{\sum\limits_d \mathbb{1}(y_d==j)} {|D|}$  
2.Solve $\hat{\beta_{jn}}$:  
$$J_2=\sum\limits_d x_{dn}log \beta_{y_d n}+\lambda_2(\sum\limits_n \beta_{jn}-1)=\sum\limits_d \sum\limits_j \mathbb{1}(y_d==j) x_{dn}log \beta_{jn}+\lambda_2(\sum\limits_n \beta_{jn}-1)$$  
$$\frac{\partial J_2}{\partial \beta_{jn}}=\frac{\sum\limits_d \mathbb{1}(y_d==j) x_{dn}}{\beta_{jn}}+\lambda_2=0\\ \Rightarrow \sum\limits_{n} \sum\limits_d \mathbb{1}(y_d==j) x_{dn}+\lambda_2 \sum\limits_n \beta_{jn}= \sum\limits_n \sum\limits_d \mathbb{1}(y_d==j) x_{dn}+\lambda_2=0 \\ \Rightarrow \lambda_2=- \sum\limits_n \sum\limits_d \mathbb{1}(y_d==j) x_{dn}$$  
So we will have: $\hat{\beta_{jn}}=-\frac{\sum\limits_d \mathbb{1}(y_d==j) x_{dn}}{\lambda_2}
=\frac{\sum\limits_d \mathbb{1}(y_d==j) x_{dn}}{\sum\limits_d \mathbb{1}(y_d==j)\sum\limits_{n^{'}} x_{dn^{'}}}$

## Problem 2: Optimization

- Implement two versions of optimization (for-loop or matrix) in the Cell 2 below. 
- Run your experiments by executing Cell 3. 
- Please do __NOT__ modify Cell 1.
- Using existing packages such as `sklearn.linear_model.LogisticRegression` is __NOT__ allowed.
- Feel free to comment the example code and write your own.
- Necessary changes outside the `TODO` blocks is fine such reformatting the output.

## Matrix Form Operation
### 1. Gradient Vector
$$\frac{\partial {log L(\beta)}}{\partial \beta}=X^{T}Y-X^{T}\frac{e^{X\beta}}{1+e^{X\beta}}$$

### 2. Hessian Matrix
$$\frac{\partial^2 {log L(\beta)}}{\partial \beta^2}= -(X^{T}p)((1-p)^TX),\text{where}\ p=\frac{e^{X\beta}}{1+e^{X\beta}}$$

In [84]:
# Cell 1: helper code. [DO NOT MODIFY]

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# data loader
def load_data():
    data = pd.read_csv("tic-tac-toe.data", sep=",")
    data.rename(columns={'x': 'top left', 'x.1': 'top middle', 
                         'x.2': 'top right','x.3': 'middle left', 
                         'o': 'middle middle', 'o.1' : 'middle right', 
                         'x.4' : 'bottom left', 'o.2' : 'bottom middle', 
                         'o.3':'bottom right','positive' : 'outcome'},inplace=True)
    data_feature = pd.get_dummies(data.iloc[:,0:9])    
    data_label = data.iloc[:,9].map({'positive': 1, 'negative': 0})
    trainX, testX, trainY, testY = train_test_split(data_feature, data_label, test_size=0.3)
    return trainX, testX, trainY, testY

In [93]:
# Cell 2: two versions of optimizer + main function

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import time
# ==================
#  Two Optimizers
# ==================

def forloop_optimizer(W, X, y, lr):
    """
    Gradient Descent (GD) optimizer implemented with for loop.
    
    Args:
        W - Parameters
        X - Features of training batch/instance
        y - Label(s) of training batch/instance
        lr - Learning rate
    
    Return:
        W_new - Updated W
    """    
    # ========= TODO start ==========
    X0 = np.ones((X.shape[0],1))
    Xnew = np.hstack((X0,X))
    dW=np.zeros(W.shape)
    for j in range(len(W)):
        for i in range (Xnew.shape[0]):
            x=Xnew[i,:]
            p=float(np.exp(dW.dot(x)))/float(1+np.exp(dW.dot(x)))
            dW[j]+=x[j]*(float(y.values[i])-p)
    W_new=W+lr*dW   
    # ========= TODO end ============
    return W_new


def matrix_optimizer(W, X, y, lr):
    """
    Gradient Descent (GD) optimizer implemented with matrix operations.
    
    See `forloop_optimizer()` for other information.
    """
    # ========= TODO start ==========
    X0 = np.ones((X.shape[0],1))
    Xnew = np.hstack((X0,X))
    dW=np.zeros(W.shape)
    P=np.exp(Xnew.dot(W))/(1+np.exp(Xnew.dot(W)))
    dW=Xnew.T.dot(y)-Xnew.T.dot(P)
    W_new=W+lr*dW
    # ========= TODO end ============ 
    return W_new

# =============================
#  Training/Testing Functions
# =============================

def evaluate(W, tstX, tstY):
    """
    Predict on test set and print out the evaluation metrics.
    """    
    # ========= TODO start ==========
    X0 = np.ones((tstX.shape[0],1))
    tstXnew = np.hstack((X0,tstX))
    predY=1*(tstXnew.dot(W)>=0)
    # Generate predY
    # ========= TODO end ============
    print("Accuracy score:")
    print(accuracy_score(tstY, predY))
    print("Confusion matrix:")
    print(confusion_matrix(tstY, predY))
    
    return accuracy_score(tstY, predY)

def logistic_regression(n_epoch,batch_size,lr,optimizer):
    """
    Logistic Regression Classifier
    
    Args:
        n_epoch: number of training epoches
        lr: learning rate
    """   
    n_features = 27
    trnX, tstX, trnY, tstY = load_data()
    W = np.random.rand(n_features + 1)
    X_indices = np.arange(trnX.shape[0])
    n_batch=int(trnX.shape[0]/batch_size)
    # Record the time before training
    for epoch in range(n_epoch):
        # ========= TODO start ==========
        for batch in range(n_batch):
            X_batch = None
            y_batch = None
        # Training Steps
        #   Step 1: iterate through batches or samples
        #   Step 2: update W by either optimizer
        #   Step 3: predict  
            batch_indices = np.random.choice(X_indices,batch_size)
        # Efficiency
        #   Print out time used for training
            X_batch = trnX.iloc[batch_indices]
            y_batch = trnY.iloc[batch_indices]
            if optimizer=='matrix':
                W=matrix_optimizer(W, X_batch, y_batch, lr)
            if optimizer=='for-loop':
                W=forloop_optimizer(W, X_batch, y_batch, lr)
        # ========= TODO end ============
        if (batch+1) % 5==0:
            evaluate(W, tstX, tstY)
    accuracy=evaluate(W, tstX, tstY)
    return accuracy

In [94]:
# Cell 3: execution
n_epoch=100
batch_size=90
lr=0.01
# Run with for-loop version optimizer
t0 = time.time()
accuracy1=logistic_regression(n_epoch,batch_size,lr,'for-loop')  # plugin your configurations
time1 = time.time()-t0
# Run with matrix-version optimizer
t0 = time.time()
accuracy2 = logistic_regression(n_epoch,batch_size,lr,'matrix')
time2 = time.time()-t0

Accuracy score:
0.7048611111111112
Confusion matrix:
[[ 19  85]
 [  0 184]]
Accuracy score:
0.9722222222222222
Confusion matrix:
[[ 93   8]
 [  0 187]]


In [95]:
# Cell 4: Result Table
data=[[time1,accuracy1],[time2,accuracy2]]
result=pd.DataFrame(data)
result.columns = ['Time','Accuracy']
result.rename(index={0:'for-loop',1:'matrix'}, inplace=True)
result

,Time,Accuracy
for-loop,13.822154,0.704861
matrix,0.345077,0.972222


## Problem 3 Poisson Regressions

Add your answers in the cell below.

### 1 $p(y;\lambda)=e^{-\lambda}\frac{\lambda^y}{y!}=\frac{1}{y!}exp[(log \lambda)y-\lambda]$  
compare with the canonical form, we have :$\eta=log \lambda,\ T(y)=y, \ a(\eta)=exp(\eta), \ b(y)=\frac{1}{y!}$

### 2 $\eta=x^{T}\beta, \mu=\lambda=exp(\eta)=exp(x^{T}\beta)$
Possion Regression : $y|x,\beta \sim Poission(exp(x^{T}\beta))$

### 3 EX:Estimate the number of traffic accident of one day based on therainfall intensity. Here the response variable is a count number and it can be any integer greater than 0, which follows the possion distribution. So the $\mu \in (0,\infty)$, which can be well explained by $exp(x^T\beta)$ other than $(x^T\beta)$ for linear regression and $\sigma(x^T\beta)$ for logistic regression

## Problem 4 Text Classification

Please read the description of this problem from the handout before diving into the actual task.

The cell(s) below are all yours. Please show the entire process with detailed comments of 

- loading/preprocessing/featurizing the dataset
- building model (you can use sklearn in this task)
- evaluating model by numerical metrics or plots

Hint:

1. Download the files. Take {`alt.atheism`, `soc.religion.christian`, `comp.graphics`, `sci.med`} as an example.
```python
>>> categories = ['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med']
>>> from sklearn.datasets import fetch_20newsgroups
>>> twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
```

2. Access the data files
```python
# Fetch training data 0.
>>> twenty_train.data[0]
'From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n'
# Fetch testing data.
>>> twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)
```

3. Access the labels
```python
# Print out labels
>>> twenty_test.target[0]
2
# Four distinct labels in total
>>> set(twenty_test.target)
{0, 1, 2, 3}
```

In [1]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Preprocessing ,loading and featurizing Data
def clean_line(t):
    return (t.replace('\n',' ')
             .replace('\r',' ')
             .replace('\t',' ')
             .replace('  ',' ')
             .strip())

def load_and_process_data():
    
    categories = ['alt.atheism',
                 'comp.graphics',
                 'comp.os.ms-windows.misc',
                 'comp.sys.ibm.pc.hardware',
                 'comp.sys.mac.hardware',
                 'comp.windows.x',
                 'misc.forsale',
                 'rec.autos',
                 'rec.motorcycles',
                 'rec.sport.baseball',
                 'rec.sport.hockey',
                 'sci.crypt',
                 'sci.electronics',
                 'sci.med',
                 'sci.space',
                 'soc.religion.christian',
                 'talk.politics.guns',
                 'talk.politics.mideast',
                 'talk.politics.misc',
                 'talk.religion.misc']
    #twenty_train = fetch_20newsgroups(subset='train', categories=categories, remove=('headers', 'footers', 'quotes'))
    twenty_train = fetch_20newsgroups(subset='train',shuffle=True,random_state=42)
    twenty_test = fetch_20newsgroups(subset='test',shuffle=True,random_state=42)
    
    X_train = [clean_line(t) for t in twenty_train.data]
    X_test = [clean_line(t) for t in twenty_test.data]
    y_train=twenty_train.target
    y_test=twenty_test.target
   
    tfidf = TfidfVectorizer()
    X_train_tfidf = tfidf.fit_transform(X_train)
    X_test_tfidf = tfidf.transform(X_test)
    return X_train_tfidf, X_test_tfidf, y_train, y_test

X_train, X_test, y_train, y_test = load_and_process_data()

In [3]:
X_train.shape

(11314, 130107)

In [98]:
# Use Multinomial Naive Bayes
nb_clf = MultinomialNB()
nb_clf.fit(X_train, y_train)
predicted_nb = nb_clf.predict(X_test)

In [99]:
# Use Logistic Regression
lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)
predicted_lr = lr_clf.predict(X_test)

In [100]:
# Use SVM Classifier
sgd_clf = SGDClassifier()
sgd_clf.fit(X_train, y_train)
predicted_svm = sgd_clf.predict(X_test)

C:\Users\shadi\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [101]:
# Confusion Matrix Comparison
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(classification_report(y_test, predicted_nb))
print(classification_report(y_test, predicted_lr))
print(classification_report(y_test, predicted_svm))

             precision    recall  f1-score   support

          0       0.80      0.52      0.63       319
          1       0.81      0.65      0.72       389
          2       0.82      0.65      0.73       394
          3       0.67      0.78      0.72       392
          4       0.86      0.77      0.81       385
          5       0.89      0.75      0.82       395
          6       0.93      0.69      0.80       390
          7       0.85      0.92      0.88       396
          8       0.94      0.93      0.93       398
          9       0.92      0.90      0.91       397
         10       0.89      0.97      0.93       399
         11       0.59      0.97      0.74       396
         12       0.84      0.60      0.70       393
         13       0.92      0.74      0.82       396
         14       0.84      0.89      0.87       394
         15       0.44      0.98      0.61       398
         16       0.64      0.94      0.76       364
         17       0.93      0.91      0.92   

In [102]:
# Accuracy Comparison 
data=[precision_recall_fscore_support(y_test, predicted_nb,average='macro')[:3],precision_recall_fscore_support(y_test, predicted_lr,average='macro')[:3],precision_recall_fscore_support(y_test, predicted_svm,average='macro')[:3]]
data=np.array(data)
data.reshape(3,3)
result=pd.DataFrame(data.T)
result.columns = ['NB','LR','SVM']
result.rename(index={0:'Precision',1:'Recall',2:'F1-score'}, inplace=True)
result

,NB,LR,SVM
Precision,0.825531,0.830127,0.849236
Recall,0.756525,0.817749,0.843811
F1-score,0.755754,0.818699,0.844019
